In [1]:
from constants import *

import os

from matplotlib import pyplot as plt
import pandas as pd

import time

import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers

# !pip install tensorflow-addons==0.8.3
# !pip install tensorflow==2.2.0-rc3
from sklearn.metrics import *
from sklearn.metrics import classification_report
import seaborn as sns

# from google.colab import drive
# drive.mount('/content/drive/')

import sys

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12177155466809741305
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2249929524
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12419157819836431406
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
device_name = sys.argv[1]  # Choose device from cmd line. Options: gpu or cpu
# shape = (int(sys.argv[2]), int(sys.argv[2]))
device_name
if device_name == "gpu":
    device_name = "/gpu:0"
else:
    device_name = "/cpu:0"

print(device_name)

/cpu:0


In [9]:
# Paths:
# in constants.py file

In [10]:
# Variables
# in constants.py file # Size of images (images are 200x200)

In [11]:
# Functions definition
#################################################################################

def training_function(base, load, weights, optimizer, loss, accuracy, model_name, train_ds, validation_ds, epochs, name_csv_history):
  '''
  - Function to train a pretrained base model
  - To the base model it is added the same classification layer, header

  Input:
      - base: base model loaded
      - load: if = 1 load with weights stored in "weights"
      - weights: weights of the model, to load if load = 1
      - optimizer: model optimizer
      - loss: loss function
      - accuracy: accuracy variable
      - model_name: text string with model name (it will be stored with this name)
      - train_ds: training dataset
      - validation_ds: validation dataset
      - epochs: number of times that the training will be runned of 100 epochs
      CANTIDAD DE VECES QUE SE EJECUTAN ENTRENAMIENTOS DE 100 ÉPOCAS
      - name_csv_history: text string with the name with which the csv with the training values will be saved, to then graph
  '''

  # contar el tiempo de ejecución
  inicio = time.time()

  # confección del modelo
  model = tf.keras.Sequential([
        base,

        tf.keras.layers.GlobalAveragePooling2D(), # se realiza un pooling a los mapas de características de la última capa del modelo base
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

  model.summary()

  # compilación del modelo
  model.compile(optimizer = optimizer,loss = loss , metrics = accuracy)

  #si load vale 1 se realiza la load de los weights
  if load == 1:
    model.set_weights(weights)

  # callback to save the best model
  callback_1 = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(MODELS_FOLDER,model_name), monitor='val_categorical_accuracy', verbose=0, save_best_only=True,
    save_weights_only=False, mode='max', save_freq='epoch'
    )

  # callback to stop the model if it hasnt improved in 50 epochs
  callback_2 = tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy', min_delta=0.001, patience=50, verbose=0, mode='max',
    baseline=None, restore_best_weights=True
    )

  # In case it is decided to train more than 100 epochs
  for i in range(epochs):

    with tf.device('/device:GPU:0'):
      h = model.fit(train_ds, epochs= 100, validation_data = validation_ds, batch_size = 2, callbacks= [callback_1, callback_2])

    #SECUENCIA PARA CONFECCIÓN DEL CSV
    if i > 0:
      hist = pd.DataFrame(h.history)
      history = pd.concat([history, hist])
    else:
      history = pd.DataFrame(h.history)

  history['epoch'] = list(range(0,len(history.index)))

  history.to_csv(os.path.join(CSV_FOLDER, name_csv_history), header=True, index=False)

  # Plot history: CE
  plt.figure()
  plt.plot(history['epoch'], history['loss'], label='Loss (training data)')
  plt.plot(history['epoch'], history['val_loss'], label='Loss (validation data)')
  plt.title('Loss')
  plt.ylabel('CE')
  plt.xlabel('Nº epoch')
  plt.legend(loc="upper left")
  plt.show()

  # Plot history: Accuracy
  plt.figure()
  plt.plot(history['epoch'], history['categorical_accuracy'], label='Accuracy (training data)')
  plt.plot(history['epoch'], history['val_categorical_accuracy'], label='Accuracy (validation data)')
  plt.title('Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Nº epoch')
  plt.legend(loc="upper left")
  plt.show()

  # show training time
  fin = time.time()
  print(round((fin - inicio)/60, 0))


#####################################################

def confusion(model, csv):
  '''
  Function to show the confusion matrix and the classification report of a given model at a csv

  Input parameters:
    - model: model to be tested
    - csv: name of the csv to test the model

  Usage example:
        model = tf.keras.models.load_model(os.path.join(dir_models, "Xcep3.h5"))

        confusion(model, 'df_test.csv')

  '''

  # read csv and create dataframe
  csvArray = np.loadtxt((os.path.join(CSV_FOLDER, csv)),dtype=str, delimiter=',', usecols=(0, 1), unpack=True)

  imgs_paths = csvArray[:,0]
  labels = csvArray[:,1]

  # test_panda = pd.read_csv(os.path.join(CSV_FOLDER, csv))

  # hago el re escalado con que he trabajado las imágenes
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

  # creo el dataset a partir del dataframe
  test_ds = test_datagen.flow_from_dataframe(
                        csvArray,
                        x_col = 'path',
                        y_col = 'clase',
                        target_size = (IMG_WIDTH, IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 32,
                        shuffle = False, # IMPORTANT to dont shuffle so labels will be ordered
                        interpolation = "bicubic"
                        )

  # variables to use
  y_true = [] # empty list to save the answers
  # labels = test_panda['clase'].unique() # class list
  unique_labels = set(labels)
  num_clases = len(unique_labels)
  dictionary = dict(zip(unique_labels, list(range(num_clases)))) # label dictionary and it correspondant number
  lista = list(imgs_paths) # list with real values

  # predicciones en número
  y_pred = np.argmax(model.predict(test_ds), axis=-1)
  # llevo las predicciones a texto para poder comparar
  for a in lista:
    y_true.append(dictionary[a])

  # Confusion Matrix
  fig, ax = plt.subplots(figsize=(7,7))
  conf_matrix = confusion_matrix(y_true, y_pred, labels=np.arange(num_clases))
  conf_matrix = conf_matrix/np.sum(conf_matrix, axis=1)
  sns.heatmap(conf_matrix, annot=True, fmt=".2f", square=True, cbar=False,
                cmap=plt.cm.jet, xticklabels=labels, yticklabels=labels,
                ax=ax)
  ax.set_ylabel('Actual')
  ax.set_xlabel('Predicted')
  ax.set_title('Confusion Matrix')
  plt.show()

  # hago el reporte de clasficación
  print('Classification Report:')
  print(classification_report(y_true, y_pred, labels=np.arange(num_clases), target_names=labels))

#### Loss function definition and evaluation function

In [12]:
# A loss object that receives the raw network output and the one-hot raw class labels is created
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# a metric object that calculates the accuracy is created
accuracy = tf.keras.metrics.CategoricalAccuracy()

# optimizer object is created
from tensorflow.keras.optimizers import *
optimizer1 = Adam(learning_rate=0.01)
optimizer2 = Adam(learning_rate=0.001)

#### Preentrenamiento test y validacion

In [13]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.
                                  )

In [14]:
test_panda = pd.read_csv(os.path.join(CSV_FOLDER, TEST_CSV))
valid_panda = pd.read_csv(os.path.join(CSV_FOLDER, VALIDATION_CSV))

In [15]:
# One hot encoding for labels
# labels_test = keras.utils.to_categorical(test_panda['class'], 3)
# keras.utils.to_categorical(valid_panda['class'], 3)

In [16]:
test_panda

,path,class
0,../../datasets/animals/final/Turtle_564_4.jpg,Turtle
1,../../datasets/animals/final/Turtle_56.jpg,Turtle
2,../../datasets/animals/final/Bear_609_3.jpg,Bear
3,../../datasets/animals/final/Bear_593_2.jpg,Bear
4,../../datasets/animals/final/Turtle_6.jpg,Turtle
...,...,...
907,../../datasets/animals/final/Chicken_479_1.jpg,Chicken
908,../../datasets/animals/final/Bear_635_4.jpg,Bear
909,../../datasets/animals/final/Turtle_56_4.jpg,Turtle
910,../../datasets/animals/final/Bear_622_2.jpg,Bear


In [18]:
def read_csv(input_csv):
    imgs_relative_path , labels = np.loadtxt(input_csv,dtype=str,
                                     delimiter=',', usecols=(0, 1), unpack=True)
    return imgs_relative_path, labels

'''test_np = read_csv(CSV_FOLDER + TEST_CSV)
np.transpose(test_np)

test_imgs = test_np[0]
test_labels = test_np[1]
test_imgs
keras.utils.to_categorical(test_labels, 3)'''

'test_np = read_csv(CSV_FOLDER + TEST_CSV)\nnp.transpose(test_np)\n\ntest_imgs = test_np[0]\ntest_labels = test_np[1]\ntest_imgs\nkeras.utils.to_categorical(test_labels, 3)'

In [19]:
test_ds = test_datagen.flow_from_dataframe(
                        test_panda,
                        x_col = 'path',
                        y_col = 'class',
                        target_size = (IMG_WIDTH,IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 32,
                        shuffle = True,
                        interpolation = "bicubic",
                        class_mode='categorical'
                        )

Found 912 validated image filenames belonging to 3 classes.


In [20]:
valid_ds = valid_datagen.flow_from_dataframe(
                        valid_panda,
                        x_col = 'path',
                        y_col = 'class',
                        target_size = (IMG_WIDTH,IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 32,
                        shuffle = True,
                        interpolation = "bicubic",
                        class_mode='categorical'
                        )

Found 911 validated image filenames belonging to 3 classes.


#### Train generator with data boost

In [21]:
train = pd.read_csv(os.path.join(CSV_FOLDER, TRAIN_CSV))
# keras.utils.to_categorical(train['class'], 3)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1./255.,
                                  rotation_range = 10,
                                  )
train_ds = train_datagen.flow_from_dataframe(
                        train,
                        x_col = 'path',
                        y_col = 'class',
                        target_size = (IMG_WIDTH,IMG_HEIGHT),
                        color_mode = 'rgb',
                        batch_size = 128,
                        shuffle = True,
                        interpolation = "bicubic",
                        class_mode='categorical'
                        )

Found 7292 validated image filenames belonging to 3 classes.


## __Xception (1) Classification Layer training. Frozen base model__

In [22]:
# base model definition to characteristic extraction
# phase 1, frozen model, adjust of classification selection

base_model = tf.keras.applications.Xception(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
                                                include_top=False,
                                                weights='imagenet')
base_model.trainable = False

In [ ]:
training_function(base_model, 0, 'NOT NECESSARY', optimizer1, loss, accuracy, "model.h5", train_ds, valid_ds, 1, "Xcep1.csv")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [36]:


  # compilación del modelo
  # model.compile(optimizer = optimizer,loss = loss , metrics = accuracy)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_20 (Dense)            (None, 128)               262272    
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                      